In [ ]:
import pandas as pd
pd.__version__

# General libraries.
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
from google.colab import drive

# this sets up a mount to google drive so we can connect to the files in our drive
# when you run this code block you will need to grant access through your berkeley account
drive.mount("/content/drive")


Mounted at /content/drive


In [ ]:
!pwd

/content


# Candidate Master

The candidate master file contains one record for each candidate who has either registered with the Federal Election Commission or appeared on a ballot list prepared by a state elections office.

In [ ]:
## Candidate Master File 2021-2022

#path = "../content/drive/MyDrive/W209 - Campaign Financing Project"
path = "../content/drive/MyDrive/W209_Campaign_Financing_Project"

candidate_master_columns = pd.read_csv(path+"/candidate_master_header.csv")
candidate_master_columns = candidate_master_columns.columns

candidate_master_df = pd.read_csv(path+'/candidate_master21_22.txt', sep="|", header=None, names = candidate_master_columns)
candidate_master_df.head()

,CAND_ID,CAND_NAME,CAND_PTY_AFFILIATION,CAND_ELECTION_YR,CAND_OFFICE_ST,CAND_OFFICE,CAND_OFFICE_DISTRICT,CAND_ICI,CAND_STATUS,CAND_PCC,CAND_ST1,CAND_ST2,CAND_CITY,CAND_ST,CAND_ZIP
0,H0AK00105,"LAMB, THOMAS",NNE,2020,AK,H,0.0,C,N,C00607515,1861 W LAKE LUCILLE DR,NaN,WASILLA,AK,99654.0
1,H0AL01055,"CARL, JERRY LEE, JR",REP,2022,AL,H,1.0,I,C,C00697789,PO BOX 852138,NaN,MOBILE,AL,36685.0
2,H0AL01063,"LAMBERT, DOUGLAS WESTLEY III",REP,2020,AL,H,1.0,O,P,C00701557,7194 STILLWATER BLVD,NaN,SPANISH FORT,AL,36527.0
3,H0AL01097,"AVERHART, JAMES",DEM,2020,AL,H,1.0,O,P,C00708867,430 SAINT LOUIS STREET,NaN,MOBILE,AL,36602.0
4,H0AL02087,"ROBY, MARTHA",REP,2020,AL,H,2.0,I,P,C00462143,NaN,NaN,MONTGOMERY,NaN,NaN


# Committee Master File 

The committee master file contains one record for each committee registered with the Federal Election Commission. This includes federal political action committees and party committees, campaign committees for presidential, house and senate candidates, as well as groups or organizations who are spending money for or against candidates for federal office.

https://www.fec.gov/campaign-finance-data/committee-master-file-description/




In [ ]:
## Committee Master File 2021-2022

committee_master_columns = pd.read_csv(path+"/committee_master_header.csv")
committee_master_columns = committee_master_columns.columns

committee_master_df = pd.read_csv(path+'/committee_master21_22.txt', sep="|", header=None, names = committee_master_columns)
committee_master_df.head()

,CMTE_ID,CMTE_NM,TRES_NM,CMTE_ST1,CMTE_ST2,CMTE_CITY,CMTE_ST,CMTE_ZIP,CMTE_DSGN,CMTE_TP,CMTE_PTY_AFFILIATION,CMTE_FILING_FREQ,ORG_TP,CONNECTED_ORG_NM,CAND_ID
0,C00000059,"HALLMARK CARDS, INC. PAC (HALLPAC)","KLEIN, CASSIE MS.","2501 MCGEE, MD853",NaN,KANSAS CITY,MO,64108,B,Q,UNK,M,C,"HALLMARK CARDS, INC.",NaN
1,C00000422,AMERICAN MEDICAL ASSOCIATION POLITICAL ACTION ...,"WALKER, KEVIN MR.","25 MASSACHUSETTS AVE, NW",SUITE 600,WASHINGTON,DC,200017400,B,Q,NaN,M,NaN,ALABAMA MEDICAL PAC,NaN
2,C00000489,D R I V E POLITICAL FUND CHAPTER 886,JERRY SIMS JR,3528 W RENO,NaN,OKLAHOMA CITY,OK,73107,U,N,NaN,Q,L,NaN,NaN
3,C00000547,KANSAS MEDICAL SOCIETY POLITICAL ACTION COMMITTEE,JERRY SLAUGHTER,623 SW 10TH AVE,NaN,TOPEKA,KS,666121627,U,Q,UNK,Q,M,KANSAS MEDICAL SOCIETY,NaN
4,C00000729,AMERICAN DENTAL ASSOCIATION POLITICAL ACTION C...,"FAIR, JULIAN H DR. III","1111 14TH STREET, NW",SUITE 1100,WASHINGTON,DC,200055627,B,Q,UNK,M,M,NaN,NaN


## Clean the zipcode column

Any zipcodes longer than 5 digits, replace with first 6 digits

In [ ]:
type(committee_master_df.CMTE_ZIP.iloc[0])

str

In [ ]:
committee_master_df.CMTE_ZIP.iloc[4][0:5]

'20005'

In [ ]:
def zipcode_fixer(zipcode):
  zip_str = str(zipcode)
  if len(zip_str) == 5:
    return zipcode
  elif len(zip_str) > 5:
    return str(zip_str[0:5])

In [ ]:
committee_master_df["CMTE_ZIP"] = committee_master_df["CMTE_ZIP"].map(zipcode_fixer)

In [ ]:
committee_master_df

,CMTE_ID,CMTE_NM,TRES_NM,CMTE_ST1,CMTE_ST2,CMTE_CITY,CMTE_ST,CMTE_ZIP,CMTE_DSGN,CMTE_TP,CMTE_PTY_AFFILIATION,CMTE_FILING_FREQ,ORG_TP,CONNECTED_ORG_NM,CAND_ID
0,C00000059,"HALLMARK CARDS, INC. PAC (HALLPAC)","KLEIN, CASSIE MS.","2501 MCGEE, MD853",NaN,KANSAS CITY,MO,64108,B,Q,UNK,M,C,"HALLMARK CARDS, INC.",NaN
1,C00000422,AMERICAN MEDICAL ASSOCIATION POLITICAL ACTION ...,"WALKER, KEVIN MR.","25 MASSACHUSETTS AVE, NW",SUITE 600,WASHINGTON,DC,20001,B,Q,NaN,M,NaN,ALABAMA MEDICAL PAC,NaN
2,C00000489,D R I V E POLITICAL FUND CHAPTER 886,JERRY SIMS JR,3528 W RENO,NaN,OKLAHOMA CITY,OK,73107,U,N,NaN,Q,L,NaN,NaN
3,C00000547,KANSAS MEDICAL SOCIETY POLITICAL ACTION COMMITTEE,JERRY SLAUGHTER,623 SW 10TH AVE,NaN,TOPEKA,KS,66612,U,Q,UNK,Q,M,KANSAS MEDICAL SOCIETY,NaN
4,C00000729,AMERICAN DENTAL ASSOCIATION POLITICAL ACTION C...,"FAIR, JULIAN H DR. III","1111 14TH STREET, NW",SUITE 1100,WASHINGTON,DC,20005,B,Q,UNK,M,M,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18405,C90021346,INTERGALACTICAL GLOBAL AND LOCAL RELIGIOUS GOV...,NaN,707 - 17TH ST APT 709,NaN,SAN DIEGO,CA,92101,U,I,NaN,Q,NaN,NaN,NaN
18406,C90021353,OPPORTUNITY FOR ALL ACTION FUND,NaN,1032 15TH STREET NW,SUITE 247,WASHINGTON,DC,20005,U,I,NaN,Q,NaN,NaN,NaN
18407,C90021361,CHARTER SCHOOLS NOW PAC,NaN,P.O. BOX 1176,NaN,AUSTIN,TX,78767,U,I,NaN,Q,NaN,NaN,NaN
18408,C90021379,ONSLOW COUNTY REPUBLICAN MEN'S CLUB,NaN,109 CORMORANT DRIVE,NaN,SWANSBORO,NC,28584,U,I,NaN,Q,NaN,NaN,NaN


# Add Extra CMTE columns

Making a summary table of total contributions from Committees to Candidates.

Step 1: Filter out rows from `committee_to_cand` with nulls for committee id, name, candidate id.

Step 2: group by `'CMTE_ID', 'NAME', 'CITY', 'STATE', 'CAND_ID'`. Aggregate sum on `TRANSACTION_AMT`.



Step 3: left join in `candidate_master_df` on `CAND_ID`.

Step 4: make new df of `committee_master_df` without `CAND_ID`, merge into our df on `'CMTE_ID'`.

Step 1:

In [ ]:
cleaned_campaign_finance_df = pd.read_csv(path+"/cleaned_campaign_finance.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
cleaned_campaign_finance_df.shape

(410514, 25)

In [ ]:
cleaned_campaign_finance_df.columns

Index(['fec_record_number', 'transaction_date', 'amendment_indicator_code',
       'amendment_indicator', 'transaction_type_code', 'transaction_type',
       'committee_id', 'comittee_name', 'committee_designation_code',
       'committee_designation', 'committee_type_code', 'committee_type',
       'connected_organization_name', 'committee_entity_type',
       'transaction_amount', 'campaign_receipt_name', 'candidate_id',
       'candidate_name', 'candidate_election_year',
       'candidate_election_office_type', 'candidate_state',
       'candidate_district', 'candidate_city', 'candidate_zip',
       'candidate_party'],
      dtype='object')

In [ ]:
cleaned_campaign_finance_df.head()

,fec_record_number,transaction_date,amendment_indicator_code,amendment_indicator,transaction_type_code,transaction_type,committee_id,comittee_name,committee_designation_code,committee_designation,...,campaign_receipt_name,candidate_id,candidate_name,candidate_election_year,candidate_election_office_type,candidate_state,candidate_district,candidate_city,candidate_zip,candidate_party
0,4012920212139007940,2020-12-24,N,New,24K,Contribution made to nonaffiliated committee,C00761528,OREGON FRONTIER PAC,D,Leadership PAC,...,GEORGIANS FOR KELLY LOEFFLER,S0GA00526,"LOEFFLER, KELLY",2020.0,S,GA,0.0,ATLANTA,30355.0,REP
1,4012920212139007941,2020-12-24,N,New,24K,Contribution made to nonaffiliated committee,C00761528,OREGON FRONTIER PAC,D,Leadership PAC,...,PERDUE FOR SENATE,S4GA11285,"PERDUE, DAVID",2022.0,S,GA,0.0,ATLANTA,30355.0,REP
2,4020320212018112012,2020-11-17,T,Termination,24K,Contribution made to nonaffiliated committee,C00710780,GRISSOM FOR U.S. SENATE,P,Principal campaign committee of a candidate,...,WARNOCK FOR GEORGIA,S0GA00559,"WARNOCK, RAPHAEL",2022.0,S,GA,0.0,ATLANTA,30355.0,DEM
3,4020320212018112003,2020-10-27,T,Termination,24K,Contribution made to nonaffiliated committee,C00710780,GRISSOM FOR U.S. SENATE,P,Principal campaign committee of a candidate,...,JAIME HARRISON FOR US SENATE,S0SC00289,"HARRISON, JAIME",2020.0,S,SC,0.0,COLUMBIA,29202.0,DEM
4,4020320212018112004,2020-11-17,T,Termination,24K,Contribution made to nonaffiliated committee,C00710780,GRISSOM FOR U.S. SENATE,P,Principal campaign committee of a candidate,...,JON OSSOFF FOR SENATE,S8GA00180,"OSSOFF, T. JONATHAN",2026.0,S,GA,0.0,ATLANTA,31145.0,DEM


In [ ]:
cleaned_campaign_finance_df = cleaned_campaign_finance_df.dropna(axis=0, subset=['committee_id', 'comittee_name', 'candidate_id','candidate_name'])
cleaned_campaign_finance_df.shape

(408179, 25)

Step 2:

In [ ]:
committee_master_df.shape 

(18410, 15)

In [ ]:
len(pd.unique(committee_master_df['CMTE_ID']))

18410

Step 4:

In [ ]:
cols_to_keep = ['CMTE_ID', 'ORG_TP',
'CMTE_ZIP',
'CMTE_CITY',
'CMTE_ST',
'CMTE_PTY_AFFILIATION']

In [ ]:
new_campaign_finance_df = pd.merge(cleaned_campaign_finance_df, committee_master_df[cols_to_keep], how='left', left_on='committee_id', right_on='CMTE_ID')

new_campaign_finance_df.head()

,fec_record_number,transaction_date,amendment_indicator_code,amendment_indicator,transaction_type_code,transaction_type,committee_id,comittee_name,committee_designation_code,committee_designation,...,candidate_district,candidate_city,candidate_zip,candidate_party,CMTE_ID,ORG_TP,CMTE_ZIP,CMTE_CITY,CMTE_ST,CMTE_PTY_AFFILIATION
0,4012920212139007940,2020-12-24,N,New,24K,Contribution made to nonaffiliated committee,C00761528,OREGON FRONTIER PAC,D,Leadership PAC,...,0.0,ATLANTA,30355.0,REP,C00761528,NaN,22314,ALEXANDRIA,VA,NaN
1,4012920212139007941,2020-12-24,N,New,24K,Contribution made to nonaffiliated committee,C00761528,OREGON FRONTIER PAC,D,Leadership PAC,...,0.0,ATLANTA,30355.0,REP,C00761528,NaN,22314,ALEXANDRIA,VA,NaN
2,4020320212018112012,2020-11-17,T,Termination,24K,Contribution made to nonaffiliated committee,C00710780,GRISSOM FOR U.S. SENATE,P,Principal campaign committee of a candidate,...,0.0,ATLANTA,30355.0,DEM,C00710780,NaN,66202,MISSION,KS,DEM
3,4020320212018112003,2020-10-27,T,Termination,24K,Contribution made to nonaffiliated committee,C00710780,GRISSOM FOR U.S. SENATE,P,Principal campaign committee of a candidate,...,0.0,COLUMBIA,29202.0,DEM,C00710780,NaN,66202,MISSION,KS,DEM
4,4020320212018112004,2020-11-17,T,Termination,24K,Contribution made to nonaffiliated committee,C00710780,GRISSOM FOR U.S. SENATE,P,Principal campaign committee of a candidate,...,0.0,ATLANTA,31145.0,DEM,C00710780,NaN,66202,MISSION,KS,DEM


In [ ]:
new_campaign_finance_df.shape

(408179, 31)

In [ ]:
new_campaign_finance_df = new_campaign_finance_df.drop('CMTE_ID', axis=1)

In [ ]:
new_campaign_finance_df.columns

Index(['fec_record_number', 'transaction_date', 'amendment_indicator_code',
       'amendment_indicator', 'transaction_type_code', 'transaction_type',
       'committee_id', 'comittee_name', 'committee_designation_code',
       'committee_designation', 'committee_type_code', 'committee_type',
       'connected_organization_name', 'committee_entity_type',
       'transaction_amount', 'campaign_receipt_name', 'candidate_id',
       'candidate_name', 'candidate_election_year',
       'candidate_election_office_type', 'candidate_state',
       'candidate_district', 'candidate_city', 'candidate_zip',
       'candidate_party', 'ORG_TP', 'CMTE_ZIP', 'CMTE_CITY', 'CMTE_ST',
       'CMTE_PTY_AFFILIATION'],
      dtype='object')

## Map outputs now

In [ ]:
columns_renamed_dict = {'ORG_TP': 'committee_organization_type_code',
 'CMTE_ZIP': 'committee_zip',
 'CMTE_CITY': 'committee_city',
 'CMTE_ST': 'committee_state',
 'CMTE_PTY_AFFILIATION': 'committee_party_affilation'}

In [ ]:
most_important_cols_df = new_campaign_finance_df.rename(columns = columns_renamed_dict)
most_important_cols_df

,fec_record_number,transaction_date,amendment_indicator_code,amendment_indicator,transaction_type_code,transaction_type,committee_id,comittee_name,committee_designation_code,committee_designation,...,candidate_state,candidate_district,candidate_city,candidate_zip,candidate_party,committee_organization_type_code,committee_zip,committee_city,committee_state,committee_party_affilation
0,4012920212139007940,2020-12-24,N,New,24K,Contribution made to nonaffiliated committee,C00761528,OREGON FRONTIER PAC,D,Leadership PAC,...,GA,0.0,ATLANTA,30355.0,REP,NaN,22314,ALEXANDRIA,VA,NaN
1,4012920212139007941,2020-12-24,N,New,24K,Contribution made to nonaffiliated committee,C00761528,OREGON FRONTIER PAC,D,Leadership PAC,...,GA,0.0,ATLANTA,30355.0,REP,NaN,22314,ALEXANDRIA,VA,NaN
2,4020320212018112012,2020-11-17,T,Termination,24K,Contribution made to nonaffiliated committee,C00710780,GRISSOM FOR U.S. SENATE,P,Principal campaign committee of a candidate,...,GA,0.0,ATLANTA,30355.0,DEM,NaN,66202,MISSION,KS,DEM
3,4020320212018112003,2020-10-27,T,Termination,24K,Contribution made to nonaffiliated committee,C00710780,GRISSOM FOR U.S. SENATE,P,Principal campaign committee of a candidate,...,SC,0.0,COLUMBIA,29202.0,DEM,NaN,66202,MISSION,KS,DEM
4,4020320212018112004,2020-11-17,T,Termination,24K,Contribution made to nonaffiliated committee,C00710780,GRISSOM FOR U.S. SENATE,P,Principal campaign committee of a candidate,...,GA,0.0,ATLANTA,31145.0,DEM,NaN,66202,MISSION,KS,DEM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408174,4061720221530344955,2022-05-23,N,New,24K,Contribution made to nonaffiliated committee,C00125732,BLACK HILLS CORPORATION POLITICAL ACTION COMMI...,B,Lobbyist/Registrant PAC,...,NE,1.0,LINCOLN,68501.0,REP,C,57709,RAPID CITY,SD,NaN
408175,4061720221530344272,2022-05-12,N,New,24K,Contribution made to nonaffiliated committee,C00010983,JOHNSON & JOHNSON POLITICAL ACTION COMMITTEE,B,Lobbyist/Registrant PAC,...,PA,1.0,LANGHORNE,19047.0,REP,C,20005,WASHINGTON,DC,NaN
408176,4061720221530344273,2022-05-13,N,New,24K,Contribution made to nonaffiliated committee,C00010983,JOHNSON & JOHNSON POLITICAL ACTION COMMITTEE,B,Lobbyist/Registrant PAC,...,PA,8.0,SCRANTON,18501.0,DEM,C,20005,WASHINGTON,DC,NaN
408177,4061720221530344274,2022-05-13,N,New,24K,Contribution made to nonaffiliated committee,C00010983,JOHNSON & JOHNSON POLITICAL ACTION COMMITTEE,B,Lobbyist/Registrant PAC,...,PA,16.0,BUTLER,16001.0,REP,C,20005,WASHINGTON,DC,NaN


In [ ]:
most_important_cols_df['committee_organization_type_code'].value_counts()

C    51379
T    23444
M    12626
L    10995
V     1500
W     1407
Name: committee_organization_type_code, dtype: int64

In [ ]:
org_type_dict = {'C':'Corporation',
                  'L':'Labor organization',
                  'M':'Membership organization',
                 'T': 'Trade association',
                 'V': 'Cooperative',
                 'W': 'Corporation without capital stock'}

In [ ]:
most_important_cols_df['committee_organization_type'] = most_important_cols_df['committee_organization_type_code']
most_important_cols_df['committee_organization_type'].replace(org_type_dict,inplace=True) 
most_important_cols_df

,fec_record_number,transaction_date,amendment_indicator_code,amendment_indicator,transaction_type_code,transaction_type,committee_id,comittee_name,committee_designation_code,committee_designation,...,candidate_district,candidate_city,candidate_zip,candidate_party,committee_organization_type_code,committee_zip,committee_city,committee_state,committee_party_affilation,committee_organization_type
0,4012920212139007940,2020-12-24,N,New,24K,Contribution made to nonaffiliated committee,C00761528,OREGON FRONTIER PAC,D,Leadership PAC,...,0.0,ATLANTA,30355.0,REP,NaN,22314,ALEXANDRIA,VA,NaN,NaN
1,4012920212139007941,2020-12-24,N,New,24K,Contribution made to nonaffiliated committee,C00761528,OREGON FRONTIER PAC,D,Leadership PAC,...,0.0,ATLANTA,30355.0,REP,NaN,22314,ALEXANDRIA,VA,NaN,NaN
2,4020320212018112012,2020-11-17,T,Termination,24K,Contribution made to nonaffiliated committee,C00710780,GRISSOM FOR U.S. SENATE,P,Principal campaign committee of a candidate,...,0.0,ATLANTA,30355.0,DEM,NaN,66202,MISSION,KS,DEM,NaN
3,4020320212018112003,2020-10-27,T,Termination,24K,Contribution made to nonaffiliated committee,C00710780,GRISSOM FOR U.S. SENATE,P,Principal campaign committee of a candidate,...,0.0,COLUMBIA,29202.0,DEM,NaN,66202,MISSION,KS,DEM,NaN
4,4020320212018112004,2020-11-17,T,Termination,24K,Contribution made to nonaffiliated committee,C00710780,GRISSOM FOR U.S. SENATE,P,Principal campaign committee of a candidate,...,0.0,ATLANTA,31145.0,DEM,NaN,66202,MISSION,KS,DEM,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408174,4061720221530344955,2022-05-23,N,New,24K,Contribution made to nonaffiliated committee,C00125732,BLACK HILLS CORPORATION POLITICAL ACTION COMMI...,B,Lobbyist/Registrant PAC,...,1.0,LINCOLN,68501.0,REP,C,57709,RAPID CITY,SD,NaN,Corporation
408175,4061720221530344272,2022-05-12,N,New,24K,Contribution made to nonaffiliated committee,C00010983,JOHNSON & JOHNSON POLITICAL ACTION COMMITTEE,B,Lobbyist/Registrant PAC,...,1.0,LANGHORNE,19047.0,REP,C,20005,WASHINGTON,DC,NaN,Corporation
408176,4061720221530344273,2022-05-13,N,New,24K,Contribution made to nonaffiliated committee,C00010983,JOHNSON & JOHNSON POLITICAL ACTION COMMITTEE,B,Lobbyist/Registrant PAC,...,8.0,SCRANTON,18501.0,DEM,C,20005,WASHINGTON,DC,NaN,Corporation
408177,4061720221530344274,2022-05-13,N,New,24K,Contribution made to nonaffiliated committee,C00010983,JOHNSON & JOHNSON POLITICAL ACTION COMMITTEE,B,Lobbyist/Registrant PAC,...,16.0,BUTLER,16001.0,REP,C,20005,WASHINGTON,DC,NaN,Corporation


In [ ]:
most_important_cols_df['committee_organization_type'].value_counts()

Corporation                          51379
Trade association                    23444
Membership organization              12626
Labor organization                   10995
Cooperative                           1500
Corporation without capital stock     1407
Name: committee_organization_type, dtype: int64

In [ ]:
most_important_cols_df.columns

Index(['fec_record_number', 'transaction_date', 'amendment_indicator_code',
       'amendment_indicator', 'transaction_type_code', 'transaction_type',
       'committee_id', 'comittee_name', 'committee_designation_code',
       'committee_designation', 'committee_type_code', 'committee_type',
       'connected_organization_name', 'committee_entity_type',
       'transaction_amount', 'campaign_receipt_name', 'candidate_id',
       'candidate_name', 'candidate_election_year',
       'candidate_election_office_type', 'candidate_state',
       'candidate_district', 'candidate_city', 'candidate_zip',
       'candidate_party', 'committee_organization_type_code', 'committee_zip',
       'committee_city', 'committee_state', 'committee_party_affilation',
       'committee_organization_type'],
      dtype='object')

In [ ]:
most_important_cols_df["candidate_zip"] = most_important_cols_df["candidate_zip"].map(zipcode_fixer)

In [ ]:
most_important_cols_df

,fec_record_number,transaction_date,amendment_indicator_code,amendment_indicator,transaction_type_code,transaction_type,committee_id,comittee_name,committee_designation_code,committee_designation,...,candidate_district,candidate_city,candidate_zip,candidate_party,committee_organization_type_code,committee_zip,committee_city,committee_state,committee_party_affilation,committee_organization_type
0,4012920212139007940,2020-12-24,N,New,24K,Contribution made to nonaffiliated committee,C00761528,OREGON FRONTIER PAC,D,Leadership PAC,...,0.0,ATLANTA,30355,REP,NaN,22314,ALEXANDRIA,VA,NaN,NaN
1,4012920212139007941,2020-12-24,N,New,24K,Contribution made to nonaffiliated committee,C00761528,OREGON FRONTIER PAC,D,Leadership PAC,...,0.0,ATLANTA,30355,REP,NaN,22314,ALEXANDRIA,VA,NaN,NaN
2,4020320212018112012,2020-11-17,T,Termination,24K,Contribution made to nonaffiliated committee,C00710780,GRISSOM FOR U.S. SENATE,P,Principal campaign committee of a candidate,...,0.0,ATLANTA,30355,DEM,NaN,66202,MISSION,KS,DEM,NaN
3,4020320212018112003,2020-10-27,T,Termination,24K,Contribution made to nonaffiliated committee,C00710780,GRISSOM FOR U.S. SENATE,P,Principal campaign committee of a candidate,...,0.0,COLUMBIA,29202,DEM,NaN,66202,MISSION,KS,DEM,NaN
4,4020320212018112004,2020-11-17,T,Termination,24K,Contribution made to nonaffiliated committee,C00710780,GRISSOM FOR U.S. SENATE,P,Principal campaign committee of a candidate,...,0.0,ATLANTA,31145,DEM,NaN,66202,MISSION,KS,DEM,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408174,4061720221530344955,2022-05-23,N,New,24K,Contribution made to nonaffiliated committee,C00125732,BLACK HILLS CORPORATION POLITICAL ACTION COMMI...,B,Lobbyist/Registrant PAC,...,1.0,LINCOLN,68501,REP,C,57709,RAPID CITY,SD,NaN,Corporation
408175,4061720221530344272,2022-05-12,N,New,24K,Contribution made to nonaffiliated committee,C00010983,JOHNSON & JOHNSON POLITICAL ACTION COMMITTEE,B,Lobbyist/Registrant PAC,...,1.0,LANGHORNE,19047,REP,C,20005,WASHINGTON,DC,NaN,Corporation
408176,4061720221530344273,2022-05-13,N,New,24K,Contribution made to nonaffiliated committee,C00010983,JOHNSON & JOHNSON POLITICAL ACTION COMMITTEE,B,Lobbyist/Registrant PAC,...,8.0,SCRANTON,18501,DEM,C,20005,WASHINGTON,DC,NaN,Corporation
408177,4061720221530344274,2022-05-13,N,New,24K,Contribution made to nonaffiliated committee,C00010983,JOHNSON & JOHNSON POLITICAL ACTION COMMITTEE,B,Lobbyist/Registrant PAC,...,16.0,BUTLER,16001,REP,C,20005,WASHINGTON,DC,NaN,Corporation


In [ ]:
most_important_cols_df[['committee_type_code', 'committee_type', 'connected_organization_name', 'committee_organization_type_code', 'committee_organization_type']]

,committee_type_code,committee_type,connected_organization_name,committee_organization_type_code,committee_organization_type
0,N,PAC - nonqualified,BENTZ,NaN,NaN
1,N,PAC - nonqualified,BENTZ,NaN,NaN
2,S,Senate,NaN,NaN,NaN
3,S,Senate,NaN,NaN,NaN
4,S,Senate,NaN,NaN,NaN
...,...,...,...,...,...
408174,Q,PAC - qualified,NaN,C,Corporation
408175,Q,PAC - qualified,NaN,C,Corporation
408176,Q,PAC - qualified,NaN,C,Corporation
408177,Q,PAC - qualified,NaN,C,Corporation


## Reorder Columns

'committee_organization_type_code', 'committee_zip',
       'committee_city', 'committee_state', 'committee_party_affilation',
       'committee_organization_type'

In [ ]:
columns_re_ordered_list = ['fec_record_number',
 'transaction_date',
 'amendment_indicator_code',
 'amendment_indicator',
 'transaction_type_code',
 'transaction_type',
 'committee_id',
 'comittee_name',
 'committee_designation_code',
 'committee_designation',
 'committee_type_code',
 'committee_type',
 'connected_organization_name',
 'committee_entity_type',
 'committee_organization_type_code', 
 'committee_organization_type',
 'committee_city', 
 'committee_state',
 'committee_zip', 
 'committee_party_affilation', 
 'transaction_amount',
 'campaign_receipt_name',
 'candidate_id',
 'candidate_name',
 'candidate_election_year',
 'candidate_election_office_type',
 'candidate_state',
 'candidate_district',
 'candidate_city',
 'candidate_zip',
 'candidate_party']

In [ ]:
final_df = most_important_cols_df.reindex(columns=columns_re_ordered_list)

In [ ]:
final_df

,fec_record_number,transaction_date,amendment_indicator_code,amendment_indicator,transaction_type_code,transaction_type,committee_id,comittee_name,committee_designation_code,committee_designation,...,campaign_receipt_name,candidate_id,candidate_name,candidate_election_year,candidate_election_office_type,candidate_state,candidate_district,candidate_city,candidate_zip,candidate_party
0,4012920212139007940,2020-12-24,N,New,24K,Contribution made to nonaffiliated committee,C00761528,OREGON FRONTIER PAC,D,Leadership PAC,...,GEORGIANS FOR KELLY LOEFFLER,S0GA00526,"LOEFFLER, KELLY",2020.0,S,GA,0.0,ATLANTA,30355,REP
1,4012920212139007941,2020-12-24,N,New,24K,Contribution made to nonaffiliated committee,C00761528,OREGON FRONTIER PAC,D,Leadership PAC,...,PERDUE FOR SENATE,S4GA11285,"PERDUE, DAVID",2022.0,S,GA,0.0,ATLANTA,30355,REP
2,4020320212018112012,2020-11-17,T,Termination,24K,Contribution made to nonaffiliated committee,C00710780,GRISSOM FOR U.S. SENATE,P,Principal campaign committee of a candidate,...,WARNOCK FOR GEORGIA,S0GA00559,"WARNOCK, RAPHAEL",2022.0,S,GA,0.0,ATLANTA,30355,DEM
3,4020320212018112003,2020-10-27,T,Termination,24K,Contribution made to nonaffiliated committee,C00710780,GRISSOM FOR U.S. SENATE,P,Principal campaign committee of a candidate,...,JAIME HARRISON FOR US SENATE,S0SC00289,"HARRISON, JAIME",2020.0,S,SC,0.0,COLUMBIA,29202,DEM
4,4020320212018112004,2020-11-17,T,Termination,24K,Contribution made to nonaffiliated committee,C00710780,GRISSOM FOR U.S. SENATE,P,Principal campaign committee of a candidate,...,JON OSSOFF FOR SENATE,S8GA00180,"OSSOFF, T. JONATHAN",2026.0,S,GA,0.0,ATLANTA,31145,DEM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408174,4061720221530344955,2022-05-23,N,New,24K,Contribution made to nonaffiliated committee,C00125732,BLACK HILLS CORPORATION POLITICAL ACTION COMMI...,B,Lobbyist/Registrant PAC,...,MIKE FLOOD FOR CONGRESS,H2NE01118,"FLOOD, MIKE",2022.0,H,NE,1.0,LINCOLN,68501,REP
408175,4061720221530344272,2022-05-12,N,New,24K,Contribution made to nonaffiliated committee,C00010983,JOHNSON & JOHNSON POLITICAL ACTION COMMITTEE,B,Lobbyist/Registrant PAC,...,BRIAN FITZPATRICK FOR ALL OF US,H6PA08277,"FITZPATRICK, BRIAN",2022.0,H,PA,1.0,LANGHORNE,19047,REP
408176,4061720221530344273,2022-05-13,N,New,24K,Contribution made to nonaffiliated committee,C00010983,JOHNSON & JOHNSON POLITICAL ACTION COMMITTEE,B,Lobbyist/Registrant PAC,...,CARTWRIGHT FOR CONGRESS,H2PA17079,"CARTWRIGHT, MATTHEW A.",2022.0,H,PA,8.0,SCRANTON,18501,DEM
408177,4061720221530344274,2022-05-13,N,New,24K,Contribution made to nonaffiliated committee,C00010983,JOHNSON & JOHNSON POLITICAL ACTION COMMITTEE,B,Lobbyist/Registrant PAC,...,MIKE KELLY FOR CONGRESS,H0PA03271,"KELLY, GEORGE J JR",2022.0,H,PA,16.0,BUTLER,16001,REP


## Save out the CSV

Saving out the CSV from above so Anand can use it in Tableau.

In [ ]:
final_df.to_csv(path+"/cleaned_campaign_finance_V2.csv",  sep="|", header=True, index=False)

In [ ]:
final_df_read = pd.read_csv(path+"/cleaned_campaign_finance_V2.csv", sep="|")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (12,14,15,19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
final_df_read.head()

,fec_record_number,transaction_date,amendment_indicator_code,amendment_indicator,transaction_type_code,transaction_type,committee_id,comittee_name,committee_designation_code,committee_designation,...,campaign_receipt_name,candidate_id,candidate_name,candidate_election_year,candidate_election_office_type,candidate_state,candidate_district,candidate_city,candidate_zip,candidate_party
0,4012920212139007940,2020-12-24,N,New,24K,Contribution made to nonaffiliated committee,C00761528,OREGON FRONTIER PAC,D,Leadership PAC,...,GEORGIANS FOR KELLY LOEFFLER,S0GA00526,"LOEFFLER, KELLY",2020.0,S,GA,0.0,ATLANTA,30355.0,REP
1,4012920212139007941,2020-12-24,N,New,24K,Contribution made to nonaffiliated committee,C00761528,OREGON FRONTIER PAC,D,Leadership PAC,...,PERDUE FOR SENATE,S4GA11285,"PERDUE, DAVID",2022.0,S,GA,0.0,ATLANTA,30355.0,REP
2,4020320212018112012,2020-11-17,T,Termination,24K,Contribution made to nonaffiliated committee,C00710780,GRISSOM FOR U.S. SENATE,P,Principal campaign committee of a candidate,...,WARNOCK FOR GEORGIA,S0GA00559,"WARNOCK, RAPHAEL",2022.0,S,GA,0.0,ATLANTA,30355.0,DEM
3,4020320212018112003,2020-10-27,T,Termination,24K,Contribution made to nonaffiliated committee,C00710780,GRISSOM FOR U.S. SENATE,P,Principal campaign committee of a candidate,...,JAIME HARRISON FOR US SENATE,S0SC00289,"HARRISON, JAIME",2020.0,S,SC,0.0,COLUMBIA,29202.0,DEM
4,4020320212018112004,2020-11-17,T,Termination,24K,Contribution made to nonaffiliated committee,C00710780,GRISSOM FOR U.S. SENATE,P,Principal campaign committee of a candidate,...,JON OSSOFF FOR SENATE,S8GA00180,"OSSOFF, T. JONATHAN",2026.0,S,GA,0.0,ATLANTA,31145.0,DEM


In [ ]:
final_df_read.shape

(408179, 31)

In [ ]:
final_df_read.dtypes

fec_record_number                     int64
transaction_date                     object
amendment_indicator_code             object
amendment_indicator                  object
transaction_type_code                object
transaction_type                     object
committee_id                         object
comittee_name                        object
committee_designation_code           object
committee_designation                object
committee_type_code                  object
committee_type                       object
connected_organization_name          object
committee_entity_type                object
committee_organization_type_code     object
committee_organization_type          object
committee_city                       object
committee_state                      object
committee_zip                         int64
committee_party_affilation           object
transaction_amount                    int64
campaign_receipt_name                object
candidate_id                    